In [1]:
import numpy as np

In [2]:
pi_x = np.load("pi_train_data_ann.npy")
pi_y = np.load("pi_train_label_ann.npy")
z_x = np.load("z_train_data_ann.npy")
z_y = np.load("z_train_label_ann.npy")

In [3]:
print np.shape(pi_x),np.shape(pi_y),np.shape(z_x),np.shape(z_y)

(9100, 1, 10, 1) (9100, 9) (9100, 1, 10, 1) (9100, 1)


In [4]:
print pi_x[600],pi_y[600],z_x[600],z_y[600]

[[[ 0.]
  [ 0.]
  [ 0.]
  [ 2.]
  [ 1.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 2.]]] [ 0.  0.  0.  0.  0.  0.  0.  0.  0.] [[[ 0.]
  [ 0.]
  [ 0.]
  [ 2.]
  [ 1.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 0.]
  [ 2.]]] [-1.]


In [4]:
def custom_loss_function(y_true, y_pred):

	p = y_pred
	pi = y_true

	zero = tf.zeros(shape = tf.shape(pi), dtype=tf.float32)
	where = tf.equal(pi, zero)

	negatives = tf.fill(tf.shape(pi), -100.0) 
	p = tf.where(where, negatives, p)

	loss = tf.nn.softmax_cross_entropy_with_logits(labels = pi, logits = p)

	return loss


In [5]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import math
import numpy as np
import keras
import tensorflow as tf
import os.path
from keras.layers import Dense, Activation, Conv2D, BatchNormalization, LeakyReLU, Flatten, merge, Input, ELU
from keras.models import load_model, Model
from keras.optimizers import SGD
from keras import regularizers

Using TensorFlow backend.


In [6]:
from keras.utils import plot_model

In [7]:
def conv_layer(_in):


    conv1 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)
    
    lr1 = ELU()(bn1)

    return lr1


def residual_layer(_in):

    conv1 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)
    
    lr1 = ELU()(bn1)
    
    conv2 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(lr1)

    bn2 = BatchNormalization()(conv2)
    
    m1 = merge([_in, bn2], mode='sum')
    
    lr2 = ELU()(m1)
    
    
    return lr2




def value_head(_in):


    conv1 = Conv2D(1, (1, 1), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)

    lr1 = ELU()(bn1)

    f1 = Flatten()(lr1)

    d1 = Dense(20, activation='linear', kernel_regularizer=regularizers.l2(0.0001))(f1)

    lr2 = ELU()(d1)

    d2 = Dense(1, use_bias=False, activation='tanh', kernel_regularizer=regularizers.l2(0.0001))(lr2)

    return d2

def policy_head(_in):


    conv1 = Conv2D(2, (1, 1), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    bn1 = BatchNormalization()(conv1)

    lr1 = ELU()(bn1)

    f1 = Flatten()(lr1)

    d1 = Dense(9, activation='linear', kernel_regularizer=regularizers.l2(0.0001))(f1)

    return d1



In [8]:
def model_z():
    inputs = Input(shape=(1, 10, 1))
    
    conv1 = conv_layer(inputs)
    
    r1 = residual_layer(conv1)
    r2 = residual_layer(r1)
    r3 = residual_layer(r2)
    r4 = residual_layer(r3)
    r5 = residual_layer(r4)
    r6 = residual_layer(r5)
    r7 = residual_layer(r6)
    r8 = residual_layer(r7)
    r9 = residual_layer(r8)
    r10 = residual_layer(r9)

    r11 = residual_layer(r10)
    r12 = residual_layer(r11)
    r13 = residual_layer(r12)
    r14 = residual_layer(r13)
    r15 = residual_layer(r14)
    r16 = residual_layer(r15)
    r17 = residual_layer(r16)
    r18 = residual_layer(r17)
    r19 = residual_layer(r18)
    r20 = residual_layer(r19)

    r21 = residual_layer(r20)
    r22 = residual_layer(r21)
    r23 = residual_layer(r22)
    r24 = residual_layer(r23)
    r25 = residual_layer(r24)
    r26 = residual_layer(r25)
    r27 = residual_layer(r26)
    r28 = residual_layer(r27)
    r29 = residual_layer(r28)
    r30 = residual_layer(r29)


    r31 = residual_layer(r30)
    r32 = residual_layer(r31)
    r33 = residual_layer(r32)
    r34 = residual_layer(r33)
    r35 = residual_layer(r34)
    r36 = residual_layer(r35)
    r37 = residual_layer(r36)
    r38 = residual_layer(r37)
    r39 = residual_layer(r38)
    r40 = residual_layer(r39)

    output = value_head(r40)



    model = Model(inputs=[inputs], outputs=[output])

    model.compile(loss='mean_squared_error', optimizer=SGD(lr=0.1, momentum = 0.9))
    
    return model


In [9]:
def model_pi():
    inputs = Input(shape=(1, 10, 1))
    
    conv1 = conv_layer(inputs)
    
    r1 = residual_layer(conv1)
    r2 = residual_layer(r1)
    r3 = residual_layer(r2)
    r4 = residual_layer(r3)
    r5 = residual_layer(r4)
    r6 = residual_layer(r5)
    r7 = residual_layer(r6)
    r8 = residual_layer(r7)
    r9 = residual_layer(r8)
    r10 = residual_layer(r9)

    r11 = residual_layer(r10)
    r12 = residual_layer(r11)
    r13 = residual_layer(r12)
    r14 = residual_layer(r13)
    r15 = residual_layer(r14)
    r16 = residual_layer(r15)
    r17 = residual_layer(r16)
    r18 = residual_layer(r17)
    r19 = residual_layer(r18)
    r20 = residual_layer(r19)

    r21 = residual_layer(r20)
    r22 = residual_layer(r21)
    r23 = residual_layer(r22)
    r24 = residual_layer(r23)
    r25 = residual_layer(r24)
    r26 = residual_layer(r25)
    r27 = residual_layer(r26)
    r28 = residual_layer(r27)
    r29 = residual_layer(r28)
    r30 = residual_layer(r29)


    r31 = residual_layer(r30)
    r32 = residual_layer(r31)
    r33 = residual_layer(r32)
    r34 = residual_layer(r33)
    r35 = residual_layer(r34)
    r36 = residual_layer(r35)
    r37 = residual_layer(r36)
    r38 = residual_layer(r37)
    r39 = residual_layer(r38)
    r40 = residual_layer(r39)

    output = policy_head(r40)



    model = Model(inputs=[inputs], outputs=[output])

    model.compile(loss=custom_loss_function, optimizer=SGD(lr=0.1, momentum = 0.9))
    
    return model


In [10]:
pi_x = np.load("pi_train_data_ann.npy")
pi_y = np.load("pi_train_label_ann.npy")

dis = np.random.choice(np.shape(pi_x)[0], 20, replace=False)

In [11]:
print dis

[ 688  214 1231  367 1359 1035  703  427 1427  864 1343  481  342  492 1384
 1616 1507  708 1688  265]


In [10]:
pi_x = np.load("pi_train_data_ann.npy")
pi_y = np.load("pi_train_label_ann.npy")


if  os.path.isfile("pi_model_ann.h5"):
        
    pi = load_model('pi_model_ann.h5', custom_objects={'custom_loss_function': custom_loss_function})
else:
    pi = model_pi()


for i in range(3000):
    
    dis = np.random.choice(np.shape(pi_x)[0], 512, replace=False)
    

    pi.fit(pi_x[dis], pi_y[dis], epochs=1, batch_size= 256)
    
pi.save("pi_model_ann.h5")

/usr/local/lib/python2.7/dist-packages/keras/engine/topology.py:1242: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


Epoch 1/1
512/512 [==============================] - 3s - loss: 1.0721     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.2835     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.8891     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.9081     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.7408     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.8617     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6893     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.7150     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6671     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6818     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6391     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.9443     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.7244     
Epoch 1/1
51

512/512 [==============================] - 0s - loss: 0.5269     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5740     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5280     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5884     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5648     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5316     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5508     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5570     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5032     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5794     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6086     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5621     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5641     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.4671     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5519     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5206     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5692     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4794     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5929     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5036     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5789     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5842     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5664     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4772     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5344     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4938     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.5490     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4715     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4922     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5038     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5399     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5015     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5545     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4650     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5076     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5365     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4753     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5128     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4595     
Epoch 1/1
512/512 [===

...

512/512 [==============================] - 0s - loss: 0.4101     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3376     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3195     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3790     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3524     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4098     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3447     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3655     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3581     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3328     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3852     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3571     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3592     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.4804     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3175     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4083     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3874     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4381     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3837     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4379     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3806     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4319     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3461     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4751     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3442     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4387     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.4175     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4367     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4133     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3953     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4403     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4111     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5169     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3761     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4441     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4031     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4910     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4368     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4346     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.6146     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4882     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4923     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4049     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5033     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3724     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4532     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5587     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5071     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6103     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.3974     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6289     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4062     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.4951     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5309     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.6082     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.4104     


In [11]:
z_x = np.load("z_train_data_ann.npy")
z_y = np.load("z_train_label_ann.npy")

if  os.path.isfile("z_model_ann.h5"):
        
    z = load_model("z_model_ann.h5")
else:
    z = model_z()


for i in range(3000):
    
    dis = np.random.choice(np.shape(z_x)[0], 512, replace=False)
    

    z.fit(z_x[dis], z_y[dis], epochs=1, batch_size= 256)
    
z.save("z_model_ann.h5")

Epoch 1/1
512/512 [==============================] - 3s - loss: 0.8028     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.5133     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.2709     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.2134     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.2111     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.2344     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.2038     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.2088     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.5014     
Epoch 1/1
512/512 [==============================] - 0s - loss: 2.0334     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.6908     
Epoch 1/1
512/512 [==============================] - 0s - loss: 2.1204     
Epoch 1/1
512/512 [==============================] - 0s - loss: 2.4319     
Epoch 1/1
51

512/512 [==============================] - 0s - loss: 1.2140     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0731     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.2135     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.3071     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1584     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1816     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1033     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1812     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.2747     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1338     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.2508     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1412     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1175     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 1.0500     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1123     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.2215     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1431     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0960     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1818     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1191     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0251     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1265     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1184     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1729     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1727     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1647     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 1.1067     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1221     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.9579     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1139     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0669     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0979     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.9727     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0819     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1598     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1753     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1204     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0108     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0888     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 1.0323     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0399     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0788     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0083     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0238     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0705     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0547     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0623     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0622     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.1557     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0384     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0616     
Epoch 1/1
512/512 [==============================] - 0s - loss: 1.0146     
Epoch 1/1
512/512 [===

...

512/512 [==============================] - 0s - loss: 0.0404     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0403     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0403     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0403     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0403     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0402     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0402     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0402     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0401     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0401     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0401     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0401     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0400     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.0371     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0371     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0370     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0370     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0370     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0369     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0369     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0369     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0369     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0368     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0368     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0368     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0367     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.0341     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0340     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0340     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0340     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0340     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0339     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0339     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0339     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0339     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0338     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0338     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0338     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0337     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.0313     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0313     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0313     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0312     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0312     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0312     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0312     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0311     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0311     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0311     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0311     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0310     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0310     
Epoch 1/1
512/512 [===

512/512 [==============================] - 0s - loss: 0.0288     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0287     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0287     
Epoch 1/1
512/512 [==============================] - 0s - loss: 0.0287     


In [12]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import math
import numpy as np
import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Activation, Conv2D, BatchNormalization, LeakyReLU, Flatten, merge, Input, ELU
from keras.models import load_model, Model
from keras.optimizers import SGD
from keras import regularizers
import os.path

def softmax_cross_entropy_with_logits(y_true, y_pred):

	p = y_pred
	pi = y_true

	zero = tf.zeros(shape = tf.shape(pi), dtype=tf.float32)
	where = tf.equal(pi, zero)

	negatives = tf.fill(tf.shape(pi), -100.0) 
	p = tf.where(where, negatives, p)

	loss = tf.nn.softmax_cross_entropy_with_logits(labels = pi, logits = p)

	return loss




class Node:


	def __init__(self,dim):
	
		self.state = np.zeros(dim*dim)
		self.actions = None
		self.n = 0
		self.w = 0
		self.q = 0
		self.v = 0
		self.p = 0
		self.parent = None
		self.child = None




def actions(n):

	n.actions = []
	
	

	for i in range(np.shape(n.state)[0]):
	
		
	
		if n.state[i] == 0:
		
			n.actions.append(i)
			
	return n
	
	
def children(n,player,opponent):

	dim = int((np.shape(n.state)[0])**(1/2.0))
	
	n.child = []

	for i in range(len(n.actions)):
	
		c = Node(dim)
		
		c.state[:] = n.state[:]
		
		c.state[n.actions[i]] = player
			
		c.parent = n	
		
		
		n.child.append(c)
		
	return n	


def conclusion(node, player, opponent, option):	

	
	
					 	
			
	diagonal_1 = int((np.shape(node.state)[0])**(1/2.0))
	
	if option == 0:
	
		k = 0
	
	
	
		for i in range(diagonal_1):
	
			sum_c = 0
	
			for j in range(diagonal_1):
		
				if node.state[k] == player:
			
					sum_c+=1
				
				k+=1	
		
			if sum_c == diagonal_1:
		
			
		
				return 1
			
		k = 0
	
	
	
		for i in range(diagonal_1):
	
			sum_c = 0
	
			for j in range(diagonal_1):
		
				if node.state[k] == opponent:
			
					sum_c+=1
				
				k+=1	
		
			if sum_c == diagonal_1:
		
				return 2		
						
	
	
	
	
	
	
		for i in range(diagonal_1):
	
			sum_r = 0
		
			k = i
	
			for j in range(diagonal_1):
		
				if node.state[k] == player:
			
					sum_r+=1
				
				k+=diagonal_1	
		
			if sum_r == diagonal_1:
		
			
		
				return 1
			

	
	
	
		for i in range(diagonal_1):
	
			sum_r = 0
		
			k = i
	
			for j in range(diagonal_1):
		
				if node.state[k] == opponent:
			
					sum_r+=1
				
				k+=diagonal_1
		
			if sum_r == diagonal_1:
		
				return 2		
						
		
		
			
		
		check  = 0
		
		sum_d1 = 0
		
		for d_1 in range(diagonal_1):
	
			if node.state[check] == player:
				
				sum_d1+=1
			check+=diagonal_1 + 1
		
		if sum_d1 == diagonal_1:
	
		
		
			return 1		
				
		
		check = 0
		
		sum_d1 = 0
		
		for d_1 in range(diagonal_1):
		
			if node.state[check] == opponent:
				
				sum_d1+=1
			check+=diagonal_1+1
		
		if sum_d1 == diagonal_1:
		
			return 2		
		
		
		check = diagonal_1-1
		
		sum_d2 = 0		
			
		for d_2 in range(diagonal_1):
		
			if node.state[check] == player:
			
				sum_d2+=1
				
			check+= diagonal_1 - 1
				
		if sum_d2 == diagonal_1:
	
		
		
			return 1
			
		
		check = diagonal_1 - 1
		
		sum_d2 = 0		
			
		for d_2 in range(diagonal_1):
		
			if node.state[check] == opponent:
			
				sum_d2+=1
				
			check+= diagonal_1 - 1
				
		if sum_d2 == diagonal_1:
		
			return 2	
			
		if 0 in node.state:
			
			return -1	
			
				 	
		
	return 0
	
			


		
def simulation(n,player,opponent,epsilon,current_move_p,pi_model,z_model):

	
	
	path = []
	ids = []
	
	
	#print "start state : ", n.state
	
	if n.child!=None:
	
	
		while n.child!=None:
	
	
		
	
	
	
	
	
			U_Q = []
		
		
			
			for i in range(len(n.child)):
		
				
		
				U_Q.append(epsilon*(n.child[i].p/float(n.child[i].n+1))+n.child[i].q)
				
			
			
					
			
			a_max = np.argmax(U_Q)
		
		
		
			ids.append(a_max)
			
			
		
			n = n.child[a_max]
		
			path.append(n)	
		
		
		
		#print "leaf state : ", n.state
		
		if conclusion(n,player,opponent,0) == -1:	
			n = actions(n)
	
			n = children(n, player, opponent)
			
            		
                
                	model = pi_model
            		
                		
			x_test = np.zeros((1,1,10))
			x_test[0,0,:9] = n.state	
			x_test[0,0,9] = player	
			x_test = x_test[...,np.newaxis]	
			p_ann = model.predict(x_test, batch_size=1)	
		
			allowed = []
		
			for i in range(len(n.state)):
		
				if n.state[i]==0:
			
					allowed.append(i)
				
				
			mask = np.ones(len(n.state),dtype=bool)
			mask[allowed] = False
		
		
			p_ann[0,mask] = -100

		

			#SOFTMAX
			odds = np.exp(p_ann)
			probs = odds / np.sum(odds) ###put this just before the for?
				
		
		
		
			model = z_model
			
			
			x_test = np.zeros((1,1,10))
			x_test[0,0,:9] = n.state	
			x_test[0,0,9] = player	
			x_test = x_test[...,np.newaxis]	
			v_ann = model.predict(x_test, batch_size=1)
	
		
	
	
			n.v = v_ann
		
		
			k = 0
		
			for i in range(len(n.state)):
		
				if i in allowed:
					n.child[k].p = probs[0,i]
				
					k+=1
				
			
		
			k = 0
		
			index = -1
			
			

			while n.parent!=None:
		
				if k==0:
	
		
					n = n.parent
					n.child[ids[index]].n+=1
					n.child[ids[index]].w+=v_ann
					n.child[ids[index]].q+=(n.child[ids[index]].w/float(n.child[ids[index]].n))
					
					
		
				
	
				else:
		
					n = n.parent
					n.child[ids[index]].n+=1
					n.child[ids[index]].w+=n.child[ids[index]].v
					n.child[ids[index]].q+=(n.child[ids[index]].w/float(n.child[ids[index]].n))
					
					
		
				k+=1
			
				index-=1
				
			#print "back up : ",n.state
	

		else:
		
			k = 0
		
			index = -1
			
			

			while n.parent!=None:
		
	
				n = n.parent
				n.child[ids[index]].n+=1
				n.child[ids[index]].w+=n.child[ids[index]].v
				n.child[ids[index]].q+=(n.child[ids[index]].w/float(n.child[ids[index]].n))
					
					
		
				k+=1
			
				index-=1
				
			#print "back up : ",n.state


		
	else:
	
		if conclusion(n,player,opponent,0) == -1:	
		
			n = actions(n)
	
			n = children(n, player, opponent)

			model = pi_model

			x_test = np.zeros((1,1,10))
			x_test[0,0,:9] = n.state	
			x_test[0,0,9] = player	
			x_test = x_test[...,np.newaxis]	
			
			p_ann = model.predict(x_test, batch_size=1)	
		
			allowed = []
		
			for i in range(len(n.state)):
		
				if n.state[i]==0:
			
					allowed.append(i)
				
				
			mask = np.ones(len(n.state),dtype=bool)
			mask[allowed] = False
		
		
			p_ann[0,mask] = -100

		

			#SOFTMAX
			odds = np.exp(p_ann)
			probs = odds / np.sum(odds) ###put this just before the for?
				
		
			model = z_model		

			x_test = np.zeros((1,1,10))
			x_test[0,0,:9] = n.state		
			x_test[0,0,9] = player
			x_test = x_test[...,np.newaxis]	
			v_ann = model.predict(x_test, batch_size=1)
	
		
	
	
		
		
			n.v = v_ann
			
			
			k = 0
		
			for i in range(len(n.state)):
		
				if i in allowed:
					n.child[k].p = probs[0,i]
				
					k+=1
					
			
			a = np.zeros(len(n.state))
				
			j = 0	
					
			for i in probs[0]:
			
				a[j] = i
				
				j+=1


			current_move_p.append(a)
				
			
							
						
			#print "current probs : ",current_move_p	
			
		
		
	return n,current_move_p

	
				

def conv_layer(_in):


	conv1 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    	bn1 = BatchNormalization()(conv1)
    	
    	lr1 = ELU()(bn1)

	return lr1
	
	
def residual_layer(_in):

    	conv1 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

    	bn1 = BatchNormalization()(conv1)
    	
    	lr1 = ELU()(bn1)
    	
    	conv2 = Conv2D(256, (3, 3), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(lr1)

    	bn2 = BatchNormalization()(conv2)
    	
    	m1 = merge([_in, bn2], mode='sum')
    	
    	lr2 = ELU()(m1)
    	
    	
    	return lr2
	
	


def value_head(_in):


	conv1 = Conv2D(1, (1, 1), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)
	
	bn1 = BatchNormalization()(conv1)
	
	lr1 = ELU()(bn1)	
	
	f1 = Flatten()(lr1)
	
	d1 = Dense(20, activation='linear', kernel_regularizer=regularizers.l2(0.0001))(f1)
	
	lr2 = ELU()(d1)
	
	d2 = Dense(1, use_bias=False, activation='tanh', kernel_regularizer=regularizers.l2(0.0001))(lr2)
	
	return d2

def policy_head(_in):


	conv1 = Conv2D(2, (1, 1), activation='linear', kernel_regularizer = regularizers.l2(0.0001),padding = 'same')(_in)

	bn1 = BatchNormalization()(conv1)
	
	lr1 = ELU()(bn1)
	
	f1 = Flatten()(lr1)
	
	d1 = Dense(9, activation='linear', kernel_regularizer=regularizers.l2(0.0001))(f1)
	
	return d1



def mcst(n, iterations,player,opponent,current_move_p,pi_model,z_model,p_val):


	
	
	z = Node(int(math.sqrt(len(n.state))))
	
	z.state = n.state
	
	
	epsilon = 1001
	
	for i in range(iterations):
	
	
		#print "Simulation : ", str(i+1), epsilon
		
		
	
		z,current_move_p = simulation(z,player,opponent,epsilon,current_move_p,pi_model,z_model)
		
		
		
		if (i+1)%10==0:
		
			epsilon-=100
		
		#for i in range(len(z.child)):
		
		#	print z.child[i].n
		
	
		
	best = []	
	
	
		
	for i in range(len(z.child)):
		
		
	
		best.append(z.child[i].n/float(100))
		
		
				
	
	
			
	p_val.append(best)
	
	return z.child[np.argmax(best)],current_move_p,p_val,np.argmax(best)
		
	
		
	

def print_maze(node):

    board = ""
    
    l = 0


    dim = int((np.shape(node.state)[0])**(1/2.0))	
   
    for i in range(int((np.shape(node.state)[0])**(1/2.0))*2):
        
        if i%2 == 0:
        
        	for k in range(int((np.shape(node.state)[0])**(1/2.0))):
            		board += "|"
            		
            		
            		
            		
            		if node.state[l]==1:
            		
            			
            		
            			board+=" A "
            			
            			board+=" "
            			
            		elif node.state[l]==2:	
            		
            			
            		
            			board+=" B "
            			
            			board+=" "
            			
            		else:	
            		
            			board+="    "
            			
            		l+=1	
            		
            		
        else:
        
        	board+= " --- "*dim
       
        board += "\n"

    return board
		
		
		
		
	 
def episode(dim,turn,pi_model,z_model,pi_model_2,z_model_2):

	states = []
	vals = []
	current_move_p = []
	free = []
	p_val = []
	p_child = []

	

	check = 0

	
	if turn == 1:
		p = 1
		
	else:
		p = 0
			
			
	n = Node(int(dim))		
		
	
	
	print "Start State  \n"
			
			
			
	maze = print_maze(n)
			
	print maze
	
	
	while conclusion(n, 1, 2, 0) == -1:

		
	
		if p==1:
		
			
		
			print "Monte Carlo 1's Move"
			
			
			a = np.zeros(len(n.state)+1)
			
			a[:9] = n.state
			
			a[9] = 1
			
			states.append(a)
			
			
			
			
			
			
			
			n,probs,p_val,best = mcst(n,101,1,2,current_move_p,pi_model,z_model,p_val)

			free.append(best)

			#print "free total : ",free

			#print "p_val : ",p_val
			#print "p_child : ",len(p_child)

			

			
			if check > 0:

				#print "p_val : ",p_val


				
			
				pi = p_val[-2][:]

			
				#print "pi : ",pi

				#print np.shape(pi),best

				#print "free : ",free[-2]

				

				v = -1

				for x in range(len(pi)):

					if pi[x]!=0:

						v+=1

					if v==free[-2]: 

						pi[x] = 0


						

				index = 0

				#print "p_val : ",p_val

				#print pi,len(p_child[-1])

				temp = []

				for i in range(int(dim)*int(dim)):

					if pi[i]!=0:

						temp.append(p_val[-1][index])

						index+=1

					else:

						temp.append(0)

					
						

				new_p = []

				for z in range(len(p_val)-1):

					new_p.append(p_val[z])

				new_p.append(temp)

				p_val = new_p

				#print "p_val : ",p_val

				#print "pi : ",pi

				
				
				

				
			
			check+=1

			p_child.append(n.child)

			#print "len : ",np.shape(p_val)

			#for i in range(len(n.child)):
			
			#	print "asd : ", probs
			
				
			
			print "After Monte Carlo 1's Move "
			
			#print "check : ", current_move_p
			
			maze = print_maze(n)
			
			print maze
		
				
		
			p=0
			
	
		elif p==0:
		
			
		
			print "Monte Carlo 2's Move"
			

			a = np.zeros(len(n.state)+1)
			
			a[:9] = n.state
			
			a[9] = 2
			
			states.append(a)
			
			
			
		
			
			
			
			
			
			
			
			n,probs,p_val,best = mcst(n,101,2,1,current_move_p,pi_model_2,z_model_2,p_val)

			free.append(best)

			#print "free total : ",free

			#print "p_val : ",p_val
			#print "p_child : ",len(p_child)

			

			
			if check > 0:

				#print "p_val : ",p_val


				
			
				pi = p_val[-2][:]

			
				#print "pi : ",pi

				#print np.shape(pi),best

				#print "free : ",free[-2]

				

				v = -1

				for x in range(len(pi)):

					if pi[x]!=0:

						v+=1

					if v==free[-2]: 

						pi[x] = 0


						

				index = 0

				#print "p_val : ",p_val

				#print pi,len(p_child[-1])

				temp = []

				for i in range(int(dim)*int(dim)):

					if pi[i]!=0:

						temp.append(p_val[-1][index])

						index+=1

					else:

						temp.append(0)

					
						

				new_p = []

				for z in range(len(p_val)-1):

					new_p.append(p_val[z])

				new_p.append(temp)

				p_val = new_p

				#print "p_val : ",p_val

				#print "pi : ",pi

				
				
				

				
			
			check+=1

			p_child.append(n.child)

			#print "len : ",np.shape(p_val)

			

			#for i in range(len(n.child)):
			
			#	print n.child[i].p,n.child[i].v
				
			
			print "After Monte Carlo 2's Move "
			
			#print "check : ", current_move_p
			
			maze = print_maze(n)
			
			print maze
		

			p=1
	
	if conclusion(n, 1, 2, 0) == 1:
	
		print "Monte Carlo 1 Wins "
		
		for i in range(len(states)):
		
			if states[i][9]==1:
		
				vals.append(1)
			else:
			
				vals.append(-1)	
		
	elif conclusion(n, 1, 2, 0) == 2:
	
		print "Monte Carlo 2 Wins"
		
		for i in range(len(states)):
			
			if states[i][9]==2:
		
				vals.append(1)
			else:
			
				vals.append(-1)	

	else:
	
		print "Game Drawn "	
		
		for i in range(len(states)):
		
			vals.append(0)
		
	
	
	
	#print "states : ",states
	
	#print "probs : ",probs
	
	#print "vals : ",vals
	
	
	#print np.shape(states)
	
	#print np.shape(probs)
	
	#print np.shape(vals)
	
	return states,probs,vals,p_val		

def main():


	inputs = Input(shape=(1, 10, 1))
	
	conv1 = conv_layer(inputs)
	
	r1 = residual_layer(conv1)
	r2 = residual_layer(r1)
	r3 = residual_layer(r2)
	r4 = residual_layer(r3)
	r5 = residual_layer(r4)
	r6 = residual_layer(r5)
	r7 = residual_layer(r6)
	r8 = residual_layer(r7)
	r9 = residual_layer(r8)
	r10 = residual_layer(r9)
	
	r11 = residual_layer(r10)
	r12 = residual_layer(r11)
	r13 = residual_layer(r12)
	r14 = residual_layer(r13)
	r15 = residual_layer(r14)
	r16 = residual_layer(r15)
	r17 = residual_layer(r16)
	r18 = residual_layer(r17)
	r19 = residual_layer(r18)
	r20 = residual_layer(r19)
	
	r21 = residual_layer(r20)
	r22 = residual_layer(r21)
	r23 = residual_layer(r22)
	r24 = residual_layer(r23)
	r25 = residual_layer(r24)
	r26 = residual_layer(r25)
	r27 = residual_layer(r26)
	r28 = residual_layer(r27)
	r29 = residual_layer(r28)
	r30 = residual_layer(r29)


	r31 = residual_layer(r30)
	r32 = residual_layer(r31)
	r33 = residual_layer(r32)
	r34 = residual_layer(r33)
	r35 = residual_layer(r34)
	r36 = residual_layer(r35)
	r37 = residual_layer(r36)
	r38 = residual_layer(r37)
	r39 = residual_layer(r38)
	r40 = residual_layer(r39)
	
	output = value_head(r40)
	
	z_model_2 = load_model("z_model_ann.h5")
	
	z_model = load_model("best_z_model_ann.h5")
	
	
	
	
	output = policy_head(r40)
	
	pi_model = load_model('best_pi_model_ann.h5', custom_objects={'custom_loss_function': custom_loss_function})
	
	
    
	pi_model_2  = load_model('pi_model_ann.h5', custom_objects={'custom_loss_function': custom_loss_function})
    
    
	turn = 0

	for ep in range(40):
	
		print "Episode : ",str(ep+1)
	
		states,_,vals,p = episode(3,turn,pi_model,z_model,pi_model_2,z_model_2)
		
		if turn == 0:
		
			turn = 1
			
		else:
		
			turn = 0	
		

if __name__ == "__main__":
    main()	

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:569: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/keras/legacy/layers.py:460: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)


Episode :  1
Start State  

|    |    |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 

Monte Carlo 2's Move
After Monte Carlo 2's Move 
|    | B  |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 

Monte Carlo 1's Move
After Monte Carlo 1's Move 
|    | B  |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 

Monte Carlo 2's Move
After Monte Carlo 2's Move 
|    | B  | B  
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 

Monte Carlo 1's Move
After Monte Carlo 1's Move 
|    | B  | B  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 

Monte Carlo 2's Move
After Monte Carlo 2's Move 
|    | B  | B  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 
| B  |    | A  
 ---  ---  --- 

Monte Carlo 1's Move
After Monte Carlo 1's Move 
| A  | B  | B  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 
| B  |    | A  
 ---  ---  --- 


After Monte Carlo 2's Move 
|    | B  |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 

Monte Carlo 1's Move
After Monte Carlo 1's Move 
|    | B  |    
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 

Monte Carlo 2's Move
After Monte Carlo 2's Move 
|    | B  | B  
 ---  ---  --- 
|    |    |    
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 

Monte Carlo 1's Move
After Monte Carlo 1's Move 
|    | B  | B  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 

Monte Carlo 2's Move
After Monte Carlo 2's Move 
|    | B  | B  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 
| B  |    | A  
 ---  ---  --- 

Monte Carlo 1's Move
After Monte Carlo 1's Move 
| A  | B  | B  
 ---  ---  --- 
|    |    | A  
 ---  ---  --- 
| B  |    | A  
 ---  ---  --- 

Monte Carlo 2's Move
After Monte Carlo 2's Move 
| A  | B  | B  
 ---  ---  --- 
| B  |    | A  
 ---  ---  --- 
| B  |    | A  
 ---  ---  --- 


KeyboardInterrupt: 